# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split    # Choose between models
from sklearn import preprocessing    # Scaling, transforming and wrangling data
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline    # Cross-validation
from sklearn.model_selection import GridSearchCV    # Cross-validation
from sklearn.metrics import mean_squared_error, r2_score    # Evaluation
from sklearn.externals import joblib    # Persist the model. Alternative to pickle. Better for large numpy arrays

# Load Data

In [2]:
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url, sep=';')    # Load from remote

In [3]:
print data.head()

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [4]:
print data.shape

(1599, 12)


In [5]:
print data.describe()

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

# Split Data into training and testing dataset

In [6]:
y = data.quality    # 'quality' is the prediction target
X = data.drop('quality', axis=1)

In [7]:
# Set aside 20% for testing. 
# Set random state for future reproduction of results. 
# Stratify the sample to make evaluation metrics more reliable.
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)

# Data Preprocessing

In [8]:
# Fit the transformer API
scaler = preprocessing.StandardScaler().fit(X_train)    # Save the mean and sd on the training dataset

In [9]:
# Applying transformer to training data
X_train_scaled = scaler.transform(X_train)
print X_train_scaled.mean(axis=0)
print X_train_scaled.std(axis=0)

[  1.16664562e-16  -3.05550043e-17  -8.47206937e-17  -2.22218213e-17
   2.22218213e-17  -6.38877362e-17  -4.16659149e-18  -2.54439854e-15
  -8.70817622e-16  -4.08325966e-16  -1.17220107e-15]
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]


In [10]:
# Applying transformer to testing data
X_test_scaled = scaler.transform(X_test)
print X_test_scaled.mean(axis=0)
print X_test_scaled.std(axis=0)

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]
[ 1.02160495  1.00135689  0.97456598  0.91099054  0.86716698  0.94193125
  1.03673213  1.03145119  0.95734849  0.83829505  1.0286218 ]


In [11]:
# Pipeline with preprocessing and model
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

# Hyperparameters to be tuned

In [12]:
print pipeline.get_params()

{'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__max_leaf_nodes': None, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__min_impurity_split': None, 'standardscaler__copy': True, 'randomforestregressor__oob_score': False, 'randomforestregressor__min_weight_fraction_leaf': 0.0, 'randomforestregressor__min_samples_split': 2, 'memory': None, 'randomforestregressor__min_impurity_decrease': 0.0, 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'randomforestregressor__random_state': None, 'standardscaler__with_mean': True, 'randomforestregressor__criterion': 'mse', 'randomforestregressor': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, ran

In [13]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

# Cross-validation Pipeline

In [14]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)    # GridSearchCV: test out all possible permutations

In [15]:
# Fit and tune model
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestregressor', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decr...mators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'randomforestregressor__max_depth': [None, 5, 3, 1], 'randomforestregressor__max_features': ['auto', 'sqrt', 'log2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
print clf.best_params_

{'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'sqrt'}


# Confirm model refit

In [17]:
print clf.refit

True


# Evaluate Model

In [18]:
y_pred = clf.predict(X_test)

In [19]:
print r2_score(y_test, y_pred)

0.458203523216


In [20]:
print mean_squared_error(y_test, y_pred)

0.349606875


# Save Model

In [21]:
joblib.dump(clf, 'rf_regressor.pkl')

['rf_regressor.pkl']

# To load the model (future)

In [22]:
#clf2 = joblib.load('rf_regressor.pkl')
#clf2.predict(X_test)     # Predict data set using loaded model